In [1]:
#from xcube_sh.geodb import get_geo_db_service
from distributed import Client

In [2]:
client = Client('ec2-52-58-160-50.eu-central-1.compute.amazonaws.com:8786')

OSError: Timed out trying to connect to 'tcp://ec2-52-58-160-50.eu-central-1.compute.amazonaws.com:8786' after 10 s: Timed out trying to connect to 'tcp://ec2-52-58-160-50.eu-central-1.compute.amazonaws.com:8786' after 10 s: connect() didn't finish in time

In [4]:
from xcube_sh.cube import open_cube
from xcube_sh.config import CubeConfig

In [33]:
x1 = 10.00  # degree
y1 = 54.27  # degree
x2 = 11.00  # degree
y2 = 54.60  # degree

bbox = x1, y1, x2, y2

In [27]:
spatial_res = 0.00018   # = 20.038 meters in degree

In [34]:
cube_config = CubeConfig(dataset_name='S2L2A',
                         band_names=['B04'],
                         tile_size=[512, 512],
                         geometry=bbox,
                         spatial_res=spatial_res,
                         time_range=['2018-05-21', '2018-05-22'],
                         time_period='1D')  

In [35]:
sh_credentials = dict(client_id='a20363fd-6733-4924-b77c-910862ef246a',
                      client_secret='sjndIW0BPRhqQ3hcU8bE',
                      instance_id='42590b08-eeb4-4c54-af0c-c87f83d15b78')

cube = open_cube(cube_config, max_cache_size=0, **sh_credentials)
cube

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 2048, lon: 5632, time: 2)
Coordinates:
  * lat        (lat) float64 54.64 54.64 54.64 54.64 ... 54.27 54.27 54.27 54.27
  * lon        (lon) float64 10.0 10.0 10.0 10.0 ... 11.01 11.01 11.01 11.01
  * time       (time) datetime64[ns] 2018-05-21T12:00:00 2018-05-22T12:00:00
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(2, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    B04        (time, lat, lon) float32 dask.array<chunksize=(1, 512, 512), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.7
    title:                     S2L2A Data Cube Subset
    history:                   [{'program': 'xcube_sh.store.SentinelHubStore'...
    date_created:              2019-11-12T16:53:40.085635
    processing_level:          L2A
    time_coverage_start:       2018-05-21T00:00:00+00:00
    time_coverage_end:         2018-05-23T00:00:00+00:00
    time_coverage_duration:    P2DT0H0M0S
    time_c

In [36]:
cube.B04.sel(time='2018-05-21T12:00:00').plot.imshow(vmin=0, vmax=0.2, cmap='Greys_r', figsize=(16, 10))

Exception: Too Many Requests, status code 429:
b'{"error":{"status":429,"reason":"Too Many Requests","message":"You have exceeded your rate limit","code":"RATE_LIMIT_EXCEEDED"}}'


## Load PSQL geodb service

In [1]:
from xcube_sh.geodb import get_geo_db_service

In [2]:
geo = get_geo_db_service(driver='pgsql', host="db-dcfs-geodb.cbfjgqxk302m.eu-central-1.rds.amazonaws.com",
            user="postgres",
            password="Oeckel6b&z")

## Find features

Find all features

In [3]:
fs = geo.find_features(collection_name='germany_sh_lakes', query="'S_NAME' = 'Hemmelmarker_See'")

['geodb_master', 'geodb_test', 'geodb_test3', 'geodb_germany_sh_lakes', 'land_use', 'geodb_germany_sh_lakes_long']


In [4]:
from IPython.display import GeoJSON

In [5]:
GeoJSON(fs)

<IPython.display.GeoJSON object>

## Use local geodb

In [7]:
geo_local = get_geo_db_service(driver='local')

In [8]:
fs = geo_local.find_feature(collection_name='germany-sh-lakes', query="S_NAME == 'Hemmelmarker_See'")

In [9]:
GeoJSON(fs)

<IPython.display.GeoJSON object>

## Create a new collection

In [10]:
fs = geo.find_features(collection_name='germany_sh_lakes', query="'S_NAME' LIKE '%'", max_records=200)

In [11]:
f = fs[0]
schema = {'properties': {}}
for k, v in f.items():
    try:
        float(f[k])
        schema['properties'][k] = 'float'
    except:
        schema['properties'][k] = 'str'

In [42]:
geo.new_collection('germany_sh_lakes_long', schema=schema)

ValueError: Collection germany_sh_lakes_long exists

In [4]:
geo.add_features(collection_name='germany_sh_lakes_long', features=fs)

'Features Added'

In [14]:
geo.find_features(collection_name='germany_sh_lakes_long', query="name LIKE '%'", fmt='gdf')

,id,properties,name,geometry,metadata_u,restricted,template,s_name,eu_cd_ls,eu_cd_lw,...,qe3162monr,qe3162qegr,qe3162val,qe33chang,qe33comp,qe33monp,qe33monr,qe33qegrou,qe33val,numreferen
0,0,"{'id': '0', 'METADATA_U': 'http://www.wasserbl...",Hemmelmarker_See,"POLYGON Z ((9.89765 54.49069 0.00000, 9.89760 ...",http://www.wasserblick.net/servlet/is/20090/?t...,1.0,Lwseggeom,Hemmelmarker_See,DE_LS_DESH_0145,DE_LW_DESH_0145,...,None,None,3.0,3.0,None,2008--2013,None,None,2.0,None
1,1,"{'id': '1', 'METADATA_U': 'http://www.wasserbl...",Langsee_Suederfahrenstedt,"POLYGON Z ((9.53135 54.57168 0.00000, 9.53138 ...",http://www.wasserblick.net/servlet/is/20090/?t...,1.0,Lwseggeom,Langsee_Suederfahrenstedt,DE_LS_DESH_0228,DE_LW_DESH_0228,...,None,None,3.0,3.0,None,2008--2013,None,None,2.0,None
2,31,"{'id': '31', 'METADATA_U': 'http://www.wasserb...",Westensee,"MULTIPOLYGON Z (((9.91504 54.29099 0.00000, 9....",http://www.wasserblick.net/servlet/is/20090/?t...,1.0,Lwseggeom,Westensee,DE_LS_DESH_0443,DE_LW_DESH_0443,...,None,None,3.0,3.0,None,2008--2013,None,None,2.0,None
3,2,"{'id': '2', 'METADATA_U': 'http://www.wasserbl...",Schaalsee_davon_Lassahner_See,"POLYGON Z ((10.94771 53.59346 0.00000, 10.9474...",http://www.wasserblick.net/servlet/is/20090/?t...,1.0,Lwseggeom,Schaalsee_davon_Lassahner_See,DE_LS_DEMV_LS_2300103,DE_LW_DEMV_2300103,...,None,None,6.0,7.0,5,2010--2014,None,None,2.0,None
4,3,"{'id': '3', 'METADATA_U': 'http://www.wasserbl...",Schoehsee,"POLYGON Z ((10.43024 54.16311 0.00000, 10.4302...",http://www.wasserblick.net/servlet/is/20090/?t...,1.0,Lwseggeom,Schoehsee,DE_LS_DESH_0359,DE_LW_DESH_0359,...,None,None,2.0,3.0,None,2008--2013,None,None,2.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,80,"{'id': '80', 'METADATA_U': 'http://www.wasserb...",Speicherbecken_Miele,"POLYGON Z ((8.98836 54.09468 0.00000, 8.98907 ...",http://www.wasserblick.net/servlet/is/20090/?t...,1.0,Lwseggeom,Speicherbecken_Miele,DE_LS_DESH_0535,DE_LW_DESH_0535,...,None,None,6.0,3.0,None,2008--2013,None,None,2.0,None
89,81,"{'id': '81', 'METADATA_U': 'http://www.wasserb...",Grosser_Binnensee,"POLYGON Z ((10.63522 54.33562 0.00000, 10.6353...",http://www.wasserblick.net/servlet/is/20090/?t...,1.0,Lwseggeom,Grosser_Binnensee,DE_LS_DESH_0109,DE_LW_DESH_0109,...,None,None,3.0,3.0,None,2008--2013,None,None,2.0,None
90,82,"{'id': '82', 'METADATA_U': 'http://www.wasserb...",Grosser_Poenitzer_See,"POLYGON Z ((10.69631 54.03813 0.00000, 10.6964...",http://www.wasserblick.net/servlet/is/20090/?t...,1.0,Lwseggeom,Grosser_Poenitzer_See,DE_LS_DESH_0115,DE_LW_DESH_0115,...,None,None,2.0,3.0,None,2008--2013,None,None,2.0,None
91,83,"{'id': '83', 'METADATA_U': 'http://www.wasserb...",Neustaedter_Binnenwasser_Ost,"MULTIPOLYGON Z (((10.79591 54.11351 0.00000, 1...",http://www.wasserblick.net/servlet/is/20090/?t...,1.0,Lwseggeom,Neustaedter_Binnenwasser_Ost,DE_LS_DESH_0284,DE_LW_DESH_0284,...,None,None,3.0,3.0,None,2008--2013,None,None,2.0,None
